# Create a PB corrected cube and transfer beam table

## Using Astropy

In [1]:
from astropy.io import fits
import numpy as np

### Opening the fits files

In [2]:
cube = '/idia/projects/mightee/mightee-hi/XMMLSS12/12/XMMLSS12_12.CORR.1310.ms.contsub.dirty.w128.image.u.piwimed.fits'
pbcube = '/idia/projects/mightee/mightee-hi/XMMLSS12/12/XMMLSS12_12.CORR.1310.ms.contsub.dirty.w128.pb.u.e.fits'
outfile = "/idia/users/aycha/pbcorrected/XMMLSS12_12.CORR.1310.ms.contsub.dirty.w128.image.u.piwimed.pbcorr.u.e.fits"

### PB correction

In [3]:
### Check the original data dimension 
hdulistout = fits.open(cube) #Opening fits file details: data + header
fitsdat = fits.getdata(cube) #data only
print(hdulistout.info())
S = fitsdat.shape #Know the data shape
if len(S) == 4:
        print("dim 4")
        nc, nf, ny, nx = S
        dim = 4
elif len(S) == 2:
        print("dim 2")
        ny, nx = S
        dim = 2
else:
        raise(Exception, "I don't know how to handle an image with this shape: "+str(S))
        
### primary beam correction
pbdat = fits.getdata(pbcube)
pbcordat = fitsdat / pbdat #divide the image and the pb file

### check the corrected data dimension
pbS = pbcordat.shape
if len(pbS) == 4:
        print("dim 4")
        pbnc, pbnf, pbny, pbnx = pbS
        ndim = 4
elif len(pbS) == 2:
        print("dim 2")
        pbny, pbnx = pbS
        ndim = 2
else:
        raise(Exception, "I don't know how to handle an image with this shape: "+str(S))
        
### Check the two datasets dimension
rescale = False
if pbnx != nx:
        rescale = True
if pbny != ny:
        rescale = True

#take the beam table
beam_table = hdulistout[1].data

#create the fits file
fits.writeto(outfile, pbcordat, header=hdulistout[0].header)
fits.append(outfile,beam_table, header=hdulistout[1].header)
print("PB corrected cube created")
print(fits.open(outfile).info())

Filename: /idia/projects/mightee/mightee-hi/XMMLSS12/12/XMMLSS12_12.CORR.1310.ms.contsub.dirty.w128.image.u.piwimed.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     137   (4096, 4096, 96, 1)   float32   
  1  BEAMS         1 BinTableHDU     30   96R x 5C   [1E, 1E, 1E, 1J, 1J]   
None
dim 4
dim 4
PB corrected cube created
Filename: /idia/users/aycha/pbcorrected/XMMLSS12_12.CORR.1310.ms.contsub.dirty.w128.image.u.piwimed.pbcorr.u.e.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     135   (4096, 4096, 96, 1)   float32   
  1  BEAMS         1 BinTableHDU     27   96R x 5C   [1E, 1E, 1E, 1J, 1J]   
None


## Using SpectralCube

* Advantages: less codes and shorter execution time if the primary beam of a specific detection is obtained.
* Drawback: 1- Cannot run if the beam or one of the cubes does not have a beam information/table i.e is a SpectralCube rather than a VaryingResolutionSpectralCube unless we have to insert the PB unit as Jy/beam. So better if the PB is directly a VaryingResolutionSpectralCube.
            2- Memory error if loading the entire cube

In [13]:
from spectral_cube import SpectralCube
import astropy.units as u

In [10]:
cube = SpectralCube.read('/idia/users/aycha/FITS/XMMLSS12_highres.1380.UVSUB.N5UVCONTSUB.NOGRID.ms.CUBE.FITS')
PB = SpectralCube.read('/idia/users/aycha/FITS/XMMLSS12_highres.1380.UVSUB.N5UVCONTSUB.NOGRID.ms.CUBE.pb.FITS')
outfile2 = "/idia/users/aycha/pbcorrected/XMMLSS12_highres.1380.UVSUB.N5UVCONTSUB.NOGRID.ms.CUBE.pbcorr_spectral_cube.FITS"

In [4]:
cube

VaryingResolutionSpectralCube with shape=(239, 4096, 4096) and unit=Jy / beam:
 n_x:   4096  type_x: RA---SIN  unit_x: deg    range:    33.319000 deg:   35.606558 deg
 n_y:   4096  type_y: DEC--SIN  unit_y: deg    range:    -5.969952 deg:   -3.694804 deg
 n_s:    239  type_s: FREQ      unit_s: Hz     range: 1380142385.889 Hz:1429881012.152 Hz

In [15]:
PB

SpectralCube with shape=(239, 4096, 4096):
 n_x:   4096  type_x: RA---SIN  unit_x: deg    range:    33.319000 deg:   35.606558 deg
 n_y:   4096  type_y: DEC--SIN  unit_y: deg    range:    -5.969952 deg:   -3.694804 deg
 n_s:    239  type_s: FREQ      unit_s: Hz     range: 1380142385.889 Hz:1429881012.152 Hz

In [30]:
c1 = cube[:,500:600,500:600] #just a try because the cube is too big
c2 = PB[:,500:600,500:600]*u.Jy/u.beam

In [31]:
cube.allow_huge_operations=True #if very big cube
#cubePB=cube/PB
cubePB = (c1/c2)*u.Jy/u.beam
print(cubePB)
#cubePB.write(outfile2, format='fits', overwrite='True')

VaryingResolutionSpectralCube with shape=(239, 100, 100) and unit=Jy / beam:
 n_x:    100  type_x: RA---SIN  unit_x: deg    range:    35.271439 deg:   35.326795 deg
 n_y:    100  type_y: DEC--SIN  unit_y: deg    range:    -5.692609 deg:   -5.637535 deg
 n_s:    239  type_s: FREQ      unit_s: Hz     range: 1380142385.889 Hz:1429881012.152 Hz


In [32]:
cubePB.beams[0] #beam table copied

Beam: BMAJ=14.723814010620117 arcsec BMIN=9.666779518127441 arcsec BPA=55.3525390625 deg

In [33]:
cubePB.sum(axis=(1,2))

<VaryingResolutionOneDSpectrum [-5.4852725e+03, 2.4929329e+01,
                                 5.9780258e+01,-1.4296558e+02,
                                -5.3425011e+01,-7.9354355e+01,
                                 1.7329407e+01,-4.9593231e+01,
                                -1.3045546e+01,-4.7221512e+01,
                                 3.1453983e+01,-2.0162018e+01,
                                 2.6663666e+00, 1.3889317e+01,
                                 1.9919151e+01,-1.8864552e+01,
                                -5.1947517e+01,-7.9612320e+01,
                                 3.5350441e+01, 4.8707844e+01,
                                 1.5042698e+01,-9.0473389e+01,
                                 8.0213989e+01,-7.8889914e+00,
                                 5.6284515e+01, 3.0106300e+01,
                                -1.7971054e+01,-6.3978662e+00,
                                 2.5532603e+00, 2.5726025e+01,
                                 5.6783772e+01, 1.23490